# Instability Analysis

In [1]:
# Load the necessary packages

import pandas as pd 
import numpy as np 
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
# import pyreadr
# import pingouin as pg
# from scipy import stats

In [2]:
# Read in the .csv file of you mvalues. Make sure CG's are in the rows and sample names are in the columns
mvals = pd.read_csv("/Users/chadpollard/Downloads/Air_Quality/Air_Quality_m_values.csv",header=0,index_col=0)
mvals.iloc[:5,:]

,203010900034_R01C01,203010900034_R02C01,203010900034_R03C01,203010900034_R04C01,203010900034_R05C01,203010900034_R06C01,203010900034_R07C01,203010900034_R08C01,203011620056_R01C01,203011620056_R02C01,...,203026840044_R07C01,203026840044_R08C01,203026840080_R01C01,203026840080_R02C01,203026840080_R03C01,203026840080_R04C01,203026840080_R05C01,203026840080_R06C01,203026840080_R07C01,203026840080_R08C01
cg18478105,-5.148756,-5.474227,-5.643913,-5.924762,-5.524155,-5.171803,-5.540871,-6.283487,-5.613538,-5.483491,...,-6.480307,-6.124570,-6.333056,-5.654344,-6.608218,-5.900278,-5.983448,-6.141624,-5.435922,-6.142097
cg09835024,-4.784730,-4.331624,-4.556301,-4.391781,-4.220033,-4.534711,-4.022329,-3.863938,-5.464822,-4.544731,...,-5.296858,-3.866696,-5.034614,-5.132442,-5.088178,-4.132668,-4.774222,-4.218043,-4.598451,-2.473514
cg14361672,-2.124001,-3.978694,-0.502530,-2.757438,-3.447125,-2.946425,-2.928630,-2.124948,-2.060511,-2.764504,...,-2.246387,-0.895692,-3.101933,-2.885770,-2.145328,-3.149136,-2.716336,-3.181564,-2.928571,0.803308
cg01763666,3.453905,3.633337,3.526783,3.491340,3.587868,3.862434,3.653217,4.853376,3.211817,3.135760,...,2.739871,3.791668,3.892276,4.034832,3.189300,4.238559,3.424108,3.702419,3.425701,4.123618
cg12950382,2.780896,3.686839,2.718345,1.640875,3.325480,3.008906,2.359139,2.480691,3.487834,2.504401,...,1.800790,2.537894,1.953974,4.173962,2.431139,2.942867,3.513544,3.702010,3.236451,-0.093237


In [3]:
# Read in the gene info data frame. Contains a list of CpG's and what promoter regions they overlap with. 
gene_info = pd.read_csv("/Volumes/Research_Data/Jenkins_Lab_Github/Instability_Code/gene_promoter_results_new_EPIC.tsv",sep='\t',header=0)
gene_info.iloc[:4,:]

,CHR,START,STOP,CG,GENE
0,chr9,117903983,117903984,cg16682686,DEC1
1,chr9,117905075,117905076,cg04761824,DEC1
2,chr9,117903424,117903425,cg26981881,DEC1
3,chr19,58857257,58857258,cg27481534,A1BG


In [4]:
# Read in the cutoffs file. This has the cutoffs for each promoter region that we used donor samples to generate.
cutoffs_5=pd.read_csv("/Volumes/Research_Data/Jenkins_Lab_Github/Instability_Code/Instability_Cutoffs.csv",header=0,index_col=0)
cutoffs_5.iloc[:]

,Mean_of_Stdev,Stdev_of_Stdev,3rd_Stdev_of_Stdev,Final_Cutoff
A2M-AS1,0.636151,0.085458,0.256373,0.892524
A2ML1,0.580846,0.132049,0.396148,0.976994
A3GALT2,0.666562,0.121330,0.363989,1.030551
AACS,0.679091,0.202159,0.606476,1.285567
AADACL3,0.659804,0.183220,0.549660,1.209464
...,...,...,...,...
ZXDB,0.887404,0.321998,0.965995,1.853398
ZYG11A,0.599069,0.149865,0.449596,1.048664
ZYG11B,1.617701,0.262081,0.786243,2.403944
ZYX,1.139518,0.204849,0.614548,1.754066


### NOTICE: After this point you shouldn't need to change the code

In [5]:
# Convert the gene promoter dataframe into a more readable dictionary
result = {}
count = 0

for index in gene_info.index:
    key = gene_info.iloc[index,4]
    value = gene_info.iloc[index,3]

    if key not in result:
        result[key] = [value]
    else:    
        result[key].append(value)

# Subset dictionary for the promoter regions that have 5 or more CpG's located in the promoter region
result_5 = {}
for key in result:
    new_length = len(result[key])
    if new_length >= 5:
        result_5[key] = result[key]
print("5+:",len(result_5))

5+: 12339


In [6]:
# Takes the mean for all samples at each promoter. We will then compare these to our training_cutoffs
instability_df = pd.DataFrame()
count = 0
for key in result_5:
    count += 1
    beta_key = mvals.loc[result[key],:]
    instability_df[key] = (np.std(beta_key))
    print("Finished", count, "/", len(result_5))
instability_df.iloc[:4,:]

Finished 1 / 12339
Finished 2 / 12339
Finished 3 / 12339
Finished 4 / 12339
Finished 5 / 12339
Finished 6 / 12339
Finished 7 / 12339
Finished 8 / 12339
Finished 9 / 12339
Finished 10 / 12339
Finished 11 / 12339
Finished 12 / 12339
Finished 13 / 12339
Finished 14 / 12339
Finished 15 / 12339
Finished 16 / 12339
Finished 17 / 12339
Finished 18 / 12339
Finished 19 / 12339
Finished 20 / 12339
Finished 21 / 12339
Finished 22 / 12339
Finished 23 / 12339
Finished 24 / 12339
Finished 25 / 12339
Finished 26 / 12339
Finished 27 / 12339
Finished 28 / 12339
Finished 29 / 12339
Finished 30 / 12339
Finished 31 / 12339
Finished 32 / 12339
Finished 33 / 12339
Finished 34 / 12339
Finished 35 / 12339
Finished 36 / 12339
Finished 37 / 12339
Finished 38 / 12339
Finished 39 / 12339
Finished 40 / 12339
Finished 41 / 12339
Finished 42 / 12339
Finished 43 / 12339
Finished 44 / 12339
Finished 45 / 12339
Finished 46 / 12339
Finished 47 / 12339
Finished 48 / 12339
Finished 49 / 12339
Finished 50 / 12339
Finished 

,A2M-AS1,A2ML1,A3GALT2,AACS,AADACL3,AADACL4,AAMDC,AANAT,AAR2,AARD,...,ZSWIM4,ZSWIM6,ZSWIM8,ZSWIM9,ZWILCH,ZXDB,ZYG11A,ZYG11B,ZYX,ZZEF1
203010900034_R01C01,0.622203,0.593005,0.592803,0.628082,0.647120,0.602577,1.520439,1.518499,0.685227,1.028624,...,0.714711,0.788018,2.116667,0.911979,0.623513,0.917038,0.585419,1.352735,0.608785,1.501789
203010900034_R02C01,0.534698,0.781441,0.489361,0.577824,0.451157,0.598569,1.165080,1.608051,0.643862,0.947152,...,0.513304,0.728413,1.756703,0.949823,0.376219,0.852322,0.518673,1.372961,1.147085,0.992558
203010900034_R03C01,1.158623,0.879379,1.327576,0.854799,0.660597,1.068818,0.854009,2.499670,2.297325,1.653095,...,1.182332,0.898418,2.727844,1.905096,0.638349,1.984514,1.350923,1.974142,0.982826,1.636330
203010900034_R04C01,0.588331,0.539141,0.304600,0.473679,0.639794,0.523230,1.247041,1.485897,0.847487,1.021523,...,0.725546,0.977552,1.405809,1.011364,0.396042,0.638969,0.465539,1.174787,1.018080,1.193138


In [7]:
# Calculating Chaos Score and total variance score
chaos_scores_df = pd.DataFrame(columns=['Sentrix_ID','Chaos_Score'])
total_variance_df = pd.DataFrame(columns=['Sentrix_ID','Total_Variance'])
for sentrix_id in  instability_df.index:
    total_variance = []
    count = 0
    for promoter in instability_df.columns:
        sample = instability_df.loc[sentrix_id,promoter]
        cutoff_3 = cutoffs_5.loc[promoter,"Final_Cutoff"]
        cutoff_3 = cutoff_3 * 1
        if sample > cutoff_3:
            variance = (sample - cutoff_3)
            total_variance.append(variance)
            count += 1
    total_variance = np.sum(total_variance)
    df2 = {'Sentrix_ID': sentrix_id, 'Chaos_Score': count}
    df3 = {'Sentrix_ID': sentrix_id, 'Total_Variance': total_variance}
    chaos_scores_df = chaos_scores_df.append(df2, ignore_index=True)
    total_variance_df = total_variance_df.append(df3, ignore_index=True)
    #print(sentrix_id, total_variance)
chaos_scores_df.index = chaos_scores_df["Sentrix_ID"]
del chaos_scores_df["Sentrix_ID"]
total_variance_df.index = total_variance_df["Sentrix_ID"]
del total_variance_df["Sentrix_ID"]
instability_results = pd.merge(chaos_scores_df,total_variance_df, left_index=True, right_index=True)
instability_results.iloc[:]

,Chaos_Score,Total_Variance
Sentrix_ID,,
203010900034_R01C01,64,8.387491
203010900034_R02C01,65,16.573491
203010900034_R03C01,3438,1099.796652
203010900034_R04C01,27,4.360601
203010900034_R05C01,41,7.105026
...,...,...
203026840080_R04C01,393,76.722218
203026840080_R05C01,400,70.620510
203026840080_R06C01,467,92.199081


# Export the results

In [8]:
# Export the results as a .csv
instability_results.to_csv("/Users/chadpollard/Desktop/Instability_Results.csv")